<a href="https://colab.research.google.com/github/AbshkPskr/Data-science-projects/blob/master/COVID_19_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analysis of coronavirus pandemic

##Collecting data
__ write about from where the data is collected
<br>__ explain data
<br>__ expain each attribute

In [15]:
import json
token = {"username":"abshkpskr","key":"7bf37d08aafdc81ecb2640a1960fddbc"}
with open('kaggle.json', 'w') as file:
    json.dump(token, file)

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sudalairajkumar/novel-corona-virus-2019-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
novel-corona-virus-2019-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
from zipfile import ZipFile
ZipFile('novel-corona-virus-2019-dataset.zip').extractall()

import os
for i in sorted(os.listdir()):
    print(i)

.config
.ipynb_checkpoints
COVID19_line_list_data.csv
COVID19_open_line_list.csv
covid_19_data.csv
kaggle.json
novel-corona-virus-2019-dataset.zip
sample_data
time_series_covid_19_confirmed.csv
time_series_covid_19_confirmed_US.csv
time_series_covid_19_deaths.csv
time_series_covid_19_deaths_US.csv
time_series_covid_19_recovered.csv


In [17]:
#libraries for data management
import numpy as np
import pandas as pd

#libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import folium 
from plotly.offline import plot
from IPython.display import HTML

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

data = pd.read_csv('covid_19_data.csv',index_col = 'SNo')
case_confirmed = pd.read_csv('time_series_covid_19_confirmed.csv')
case_deaths = pd.read_csv('time_series_covid_19_deaths.csv')
case_recovered = pd.read_csv('time_series_covid_19_recovered.csv')

In [0]:
data = data.drop('Last Update',axis = 1)
data = data.fillna('unknown')

data['Active'] = data['Confirmed'] - (data['Deaths'] + data['Recovered'])
data['Mortality Rate'] = (data['Deaths']/data['Confirmed'])*100

In [0]:
case_data = pd.DataFrame()

country_state = data.groupby(['Country/Region','Province/State']).size().reset_index(name = 'a').drop('a',1)

for country,state in country_state.values:
    temp = data[(data['Country/Region'] == country) & (data['Province/State'] == state)]
    temp1 = pd.DataFrame()
    temp1  = temp1.append(temp.iloc[0:1,:],ignore_index=True)
    temp1['new_confirmed'] = 0.0
    temp1['new_deaths'] = 0.0
    temp1['new_recovered'] = 0.0
    for i in range(1,len(temp)):
        date,sta,ctr,confirmed,deaths,recovered,active,mrate  = temp.iloc[i].values
        new_confirmed = temp.iloc[i,3] - temp.iloc[i-1,3]
        new_deaths = temp.iloc[i,4] - temp.iloc[i-1,4]
        new_recovered = temp.iloc[i,5] - temp.iloc[i-1,5]
        if new_confirmed < 0:
            confirmed = temp.iloc[i-1][3]
            new_confirmed = 0
        if new_deaths < 0:
            deaths = temp.iloc[i-1][4]
            new_deaths = 0
        if new_recovered < 0:
            recovered = temp.iloc[i-1][5]
            new_recovered = 0
        temp1.loc[i] = [date,sta,ctr,confirmed,deaths,recovered,active,mrate,new_confirmed,new_deaths,new_recovered]
    case_data = pd.concat([case_data,temp1],ignore_index=True)

#case_data


In [0]:
case_data['ObservationDate'] = pd.to_datetime(case_data['ObservationDate'])
case_data['Confirmed']  = case_data['Confirmed'].astype('int')
case_data['Deaths']  = case_data['Deaths'].astype('int')
case_data['Recovered']  = case_data['Recovered'].astype('int')
case_data['Active']  = case_data['Active'].astype('int')
case_data['new_confirmed']  = case_data['new_confirmed'].astype('int')
case_data['new_deaths']  = case_data['new_deaths'].astype('int')
case_data['new_recovered']  = case_data['new_recovered'].astype('int')

##Data analysis
__ tell story with data as the whole 
<br>__ draw graphs and charts to extract insights.
<br>__ do indepth analysis 
<br>__ explain different terms like mortality rate, death rate, crude mortality rate etc.


###General Analysis

In [0]:
country_case_data = case_data.groupby(['Country/Region','ObservationDate'])['Confirmed','Deaths','Recovered','Active'].sum().reset_index()
country_case_data = country_case_data.groupby('Country/Region')['Confirmed','Deaths','Recovered','Active'].max()

country_case_data.sort_values('Confirmed', ascending= False).style.background_gradient(cmap='Blues',subset=["Confirmed"])\
.background_gradient(cmap='Reds',subset=["Deaths"])\
.background_gradient(cmap='Greens',subset=["Recovered"])\
.background_gradient(cmap='Purples',subset=["Active"])

In [98]:
from bokeh.io import output_file, show, output_notebook,curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, layouts, BoxAnnotation


def tooltip(attr,color):
    return """
    <div style ="background-color:"""+ color +""";">         
        <div>
            <span style="font-size: 15px; color: white;font-family:century gothic;">@""" + attr + """{0,000} """ + attr + """</span>
        </div>
    </div>"""

world_cumulative = pd.DataFrame(case_data.groupby(['ObservationDate']).sum())

curdoc().theme = 'caliber'
plot = figure(title='Population Graph of India and Pakistan', x_axis_label='Month', y_axis_label='Number of cases in Lakhs',
              x_axis_type='datetime',y_axis_type='mercator',
              sizing_mode='stretch_both', plot_height=500)#, plot_width=980)
p1 = plot.line(x='ObservationDate', y = 'Confirmed', line_width=3, line_color='blue', legend_label='Confirmed',source = world_cumulative)
p2 = plot.line(x='ObservationDate', y = 'Deaths', line_width=3, line_color='red', legend_label='Deaths',source = world_cumulative)
p3 = plot.line(x='ObservationDate', y = 'Recovered', line_width=3, line_color='green', legend_label='Recovered',source = world_cumulative)
p4 = plot.line(x='ObservationDate', y = 'Active', line_width=3, line_color='purple', legend_label='Active',source = world_cumulative)
plot.add_tools(HoverTool(renderers=[p2], tooltips=tooltip('Deaths','red'),mode='vline'))
plot.add_tools(HoverTool(renderers=[p3], tooltips=tooltip('Recovered','green'),mode='vline'))
plot.add_tools(HoverTool(renderers=[p4], tooltips=tooltip('Active','purple'),mode='vline'))
plot.add_tools(HoverTool(renderers=[p1], tooltips=tooltip('Confirmed','blue'),mode='vline'))
plot.legend.location = 'top_left'
plot.left[0].formatter.use_scientific = False
low_box = BoxAnnotation(fill_color="Blue")
plot.add_layout(low_box)

output_notebook(show(plot))

In [69]:
world_cumulative = pd.DataFrame(case_data.groupby(['ObservationDate']).sum())

fig = go.Figure()
fig.add_trace(go.Scatter(x=world_cumulative.index,y=world_cumulative['Confirmed'],mode='lines',name='Confirmed',line=dict( width=4)))
fig.add_trace(go.Scatter(x=world_cumulative.index,y=world_cumulative['Deaths'],mode='lines',name='Deaths',line=dict( width=4)))
fig.add_trace(go.Scatter(x=world_cumulative.index,y=world_cumulative['Recovered'],mode='lines',name='Recovered',line=dict( width=4)))
fig.add_trace(go.Scatter(x=world_cumulative.index,y=world_cumulative['Active'],mode='lines',name='Active',line=dict( width=4)))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=30,pad=0),paper_bgcolor="white",)
fig.layout.hovermode = 'x'
fig.show()

In [74]:
world_daily_cumulative = pd.DataFrame(case_data.groupby(['ObservationDate']).sum())

fig = go.Figure()
fig.add_trace(go.Scatter(x=world_daily_cumulative.index,y=world_daily_cumulative['new_confirmed'],mode='lines',name='Confirmed',line=dict( width=4)))
fig.add_trace(go.Scatter(x=world_daily_cumulative.index,y=world_daily_cumulative['new_deaths'],mode='lines',name='Deaths',line=dict( width=4)))
fig.add_trace(go.Scatter(x=world_daily_cumulative.index,y=world_daily_cumulative['new_recovered'],mode='lines',name='Recovered',line=dict( width=4)))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=30,pad=0),paper_bgcolor="white",)
fig.layout.hovermode = 'x'
fig.show()

###World Map Visualization

In [0]:
def CreateMap(data,color,fill_color):
    _map = folium.Map(location=[45,0], tiles="Stamen Toner", zoom_start=3)
    for name,cases,deaths,lat,lon in data.values:
        folium.CircleMarker([lat,lon],radius=((int(np.log(cases + 1)))*1.5),color=color,fill_color=fill_color,
                            tooltip = "<h5 style='text-align:center;font-weight: bold'>"+ name +"</h5>"+
                            "<hr style='margin:10px;'>"+
                            "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                            "<li>Confirmed: "+str(cases)+"</li>"+
                            "<li>Deaths:   "+str(deaths)+"</li>"+
                            "</ul>",fill_opacity=0.7).add_to(_map)
    return _map

df_map_plot = pd.DataFrame()
for i in case_confirmed.index:
    if pd.isna(case_confirmed.loc[i,'Province/State']):
        df_map_plot.loc[i,'location'] = case_confirmed.loc[i,'Country/Region']
    else:
        df_map_plot.loc[i,'location'] = str(case_confirmed.loc[i,'Province/State']) + ", " + case_confirmed.loc[i,'Country/Region']
    if case_confirmed.loc[i,case_confirmed.columns[-1]] < 0 :
        df_map_plot.loc[i,'cases'] = 1
    else:
        df_map_plot.loc[i,'cases'] = case_confirmed.loc[i,case_confirmed.columns[-1]]
        df_map_plot.loc[i,'deaths'] = case_deaths.loc[i,case_deaths.columns[-1]]
    df_map_plot.loc[i,'Lat'] = str(case_confirmed.loc[i,'Lat'])
    df_map_plot.loc[i,'Long'] = str(case_confirmed.loc[i,'Long'])

df_map_plot['Lat'] = df_map_plot['Lat'].astype('float')
df_map_plot['Long'] = df_map_plot['Long'].astype('float')
df_map_plot['cases'] = df_map_plot['cases'].astype('int')
df_map_plot['deaths'] = df_map_plot['deaths'].astype('int')

CreateMap(df_map_plot,'#022474','#4D76D7')


###Race Map Visualization

In [0]:
case_confirmed_racemap = pd.DataFrame(case_confirmed.groupby('Country/Region').sum()).reset_index()
case_confirmed_racemap.loc[case_confirmed_racemap['Country/Region'] == 'US','Country/Region'] = 'United States'
case_confirmed_racemap.loc[case_confirmed_racemap['Country/Region'] == 'Korea, South','Country/Region'] = 'South Korea'
for col in case_confirmed_racemap.columns[3:]:
    case_confirmed_racemap.rename(columns = {col:str(pd.to_datetime(col))[0:10]},inplace = True)

flags = pd.read_csv('https://raw.githubusercontent.com/AbshkPskr/Portfolio/master/Country_Flags.csv')
flags = flags.drop('Images File Name',1).rename(columns={'Country':'Country/Region'})
confirmed_with_flag = case_confirmed_racemap.merge(flags,on='Country/Region',how='left')
confirmed_with_flag.to_csv('country.csv')
files.download('country.csv')

In [0]:
%%HTML
<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/2059674" data-url="https://flo.uri.sh/visualisation/2059674/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>

###Comparison between most effected countries 

Countries with more than 1 lakh cases

In [0]:
case_data['ObservationDate'].min()

Timestamp('2020-01-22 00:00:00')

In [0]:
def CreateComparisonPlot(attr):
    case_country = case_data.groupby(['Country/Region','ObservationDate']).sum().reset_index()
    countries = case_country[case_country['Confirmed'] > 100000]['Country/Region'].unique()
    fig = go.Figure()
    for i in countries:
        one_country = case_country[(case_country['Country/Region'] == i)][['ObservationDate',attr]]
        fig.add_trace(go.Scatter(x=case_country.index,
                                y=one_country[attr],mode='lines',
                                name=i,line=dict( width=4)))
    
    # fig.add_annotation(text="First case of United states",
    #                    x='2020-01-23', y=1, arrowhead=0, showarrow=True)
    fig.update_layout(margin=dict(l=0,r=0,b=0,t=0,pad=0),paper_bgcolor="white")#, yaxis_type="log")
    fig.layout.hovermode = 'x'
    fig.show()

In [0]:
CreateComparisonPlot('Confirmed')

In [0]:
dCreateComparisonPlot('Deaths')

In [0]:
CreateComparisonPlot('Recovered')

In [0]:
CreateComparisonPlot('new_confirmed')

In [0]:
CreateComparisonPlot('new_deaths')

In [0]:
CreateComparisonPlot('new_recovered')

###Testing Analysis

In [0]:
tests_data = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv')

In [0]:
tests_data.drop(tests_data[tests_data["Entity"] == "India - people tested"].index,inplace=True)
tests_data.drop(tests_data[tests_data["Entity"] == "United Kingdom - people tested"].index,inplace=True)
tests_data.drop(tests_data[tests_data["Entity"] == "Singapore - people tested"].index,inplace=True)
tests_data.drop(tests_data[tests_data["Entity"] == "United States - specimens tested (CDC)"].index,inplace=True)

In [0]:
tests_data['Country'] = [i.split(" - ")[0] for i in tests_data['Entity'].values]
tests_data['Cumulative total per million'] = tests_data['Cumulative total per thousand']*1000

In [0]:
tests_data = tests_data.groupby('Country')['Cumulative total','Cumulative total per million'].max().reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [0]:
con = pd.DataFrame(case_confirmed.groupby('Country/Region')[case_confirmed.columns[-1]].sum()).reset_index()
con = con.rename(columns = {'Country/Region':'Country',case_confirmed.columns[-1]:'Confirmed'})
con.loc[(con['Country'] == 'Korea, South'),'Country'] = 'South Korea'
con.loc[(con['Country'] == 'US'),'Country'] = 'United States'
con.loc[(con['Country'] == 'Taiwan*'),'Country'] = 'Taiwan'
tests_data = tests_data.merge(con,how = 'left', on = 'Country')

In [0]:
det = pd.DataFrame(case_deaths.groupby('Country/Region')[case_confirmed.columns[-1]].sum()).reset_index()
det = det.rename(columns = {'Country/Region':'Country',case_confirmed.columns[-1]:'Deaths'})
det.loc[(det['Country'] == 'Korea, South'),'Country'] = 'South Korea'
det.loc[(det['Country'] == 'US'),'Country'] = 'United States'
det.loc[(det['Country'] == 'Taiwan*'),'Country'] = 'Taiwan'
tests_data = tests_data.merge(det,how = 'left', on = 'Country')

In [0]:
tests_data['Mortality Rate'] = (tests_data['Deaths']/tests_data['Confirmed'])*100
tests_data['Spread Rate'] = (tests_data['Confirmed']/tests_data['Cumulative total'])*100
tests_data = tests_data.sort_values('Cumulative total',ascending=False).reset_index().drop('index',axis = 1).dropna()
tests_data['Cumulative total per million'] = tests_data['Cumulative total per million'].astype('int')
tests_data['Confirmed'] = tests_data['Confirmed'].astype('int')
tests_data['Deaths'] = tests_data['Deaths'].astype('int')

In [0]:
tests_data[tests_data['Cumulative total'] > 200000].style.background_gradient(cmap='BuGn',subset=['Cumulative total'])\
.background_gradient(cmap='bone_r',subset=['Cumulative total per million'])\
.background_gradient(cmap='Blues',subset=['Confirmed'])\
.background_gradient(cmap='Reds',subset=['Deaths'])\
.background_gradient(cmap='YlOrBr',subset=['Mortality Rate'])\
.background_gradient(cmap='Purples',subset=['Spread Rate'])

In [0]:
tests = tests_data.head(40)

fig = go.Figure()
fig.add_trace(go.Bar(x=tests['Country'],y=tests['Cumulative total'],
                     text=tests['Confirmed'],name='Confirmed',marker= { 'color': 'rgb(47,138,0)'}))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=30,pad=0),paper_bgcolor="white",)
fig.layout.hovermode = 'x'
fig.show()

In [0]:
tests_per_million = tests_data.sort_values('Cumulative total per million',ascending = False).head(40)

fig = go.Figure()
fig.add_trace(go.Bar(x=tests_per_million['Country'],y=tests_per_million['Cumulative total per million'],
                     name='Confirmed',marker= { 'color': 'rgb(105,67,144)'}))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=30,pad=0),paper_bgcolor="white",)
fig.layout.hovermode = 'x'
fig.show()

In [0]:
tests_comparison = tests_data.head(40)

fig = go.Figure()
fig = go.Figure()
fig.add_trace(go.Bar(x=tests_comparison['Country'],y=tests['Cumulative total'],
                     name = 'Total Tests',marker= { 'color': 'rgb(47,138,0)'}))
fig.add_trace(go.Bar(x=tests_comparison['Country'],y=tests['Cumulative total per million'],
                     name = 'Tests per million',marker= { 'color': 'rgb(105,67,144)'}))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=30,pad=0),paper_bgcolor="white",yaxis_type="log")
fig.layout.hovermode = 'x'
fig.show()

In [0]:
tests_mortality = tests_data.sort_values('Mortality Rate',ascending = False).head(40)


fig = go.Figure()
fig = go.Figure()
fig.add_trace(go.Bar(x=tests_mortality['Country'],y=tests_mortality['Mortality Rate'],
                     name = 'Total Tests',marker= { 'color': 'rgb(183,187,0)'}))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=30,pad=0),paper_bgcolor="white")
fig.layout.hovermode = 'x'
fig.show()

In [0]:
tests_spread = tests_data.sort_values('Spread Rate',ascending = False).head(40)


fig = go.Figure()
fig = go.Figure()
fig.add_trace(go.Bar(x=tests_spread['Country'],y=tests_spread['Spread Rate'],
                     name = 'Total Tests',marker= { 'color': 'rgb(185,70,188)'}))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=30,pad=0),paper_bgcolor="white")
fig.layout.hovermode = 'x'
fig.show()

###Spread rate comparison

In [0]:
import requests
from bs4 import BeautifulSoup
url = "https://www.worldometers.info/coronavirus/"
webpage = requests.get(url)
webpage.text
soup = BeautifulSoup(webpage.text, 'html.parser')
# print(soup.prettify())

myTable = soup.find('table')

flag = False
table = []
for i in myTable.findAll('td'):
    if i.text == 'Total:':
        flag = False
    if flag == True:
        table.append(i.text)
    if i.text == 'All':
        flag = True

country = []
total_cases = []
total_tests = []
spread_rate = []

for i in range(0,len(table),13):
    try:
        case =int(float(table[i+1].replace(',', '')))
    except:
        case = 0
    try:
        test =int(float(table[i+10].replace(',', '')))
    except:
        test = 0
    try:
        rate = case/test
    except:
        rate = np.nan
    if rate > 1: continue
    #if case < 50000: continue
    country.append(table[i])
    total_cases.append(case)
    total_tests.append(test)
    spread_rate.append(rate)

country_tests = pd.DataFrame({'country':country,'total_cases':total_cases,
                              'total_tests':total_tests,'spread_rate':spread_rate})

In [0]:
country_tests = country_tests.sort_values('spread_rate',ascending=False)
fig = go.Figure().add_trace(go.Bar(x=country_tests['country'],y=country_tests['spread_rate'],marker_color='rgba(0, 41, 154, .8)'))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=50,pad=0),paper_bgcolor="white",title_text = 'Number of CoronaVirus Cases worldwide',)
fig.layout.hovermode = 'x'
fig.show()

###Age Analysis

In [0]:
covid_data = pd.read_csv("COVID19_line_list_data.csv")

In [0]:
covid_data['death'] = [0 if i == '0' else 1 for i in covid_data.death]

In [0]:
for i in covid_data.index:
    age = covid_data.loc[covid_data.index == i,'age']
    try:
        age = int(age)
    except:
        continue
    if age <= 10: covid_data.loc[covid_data.index == i,'age'] = '1-10'
    if 11 <= age <= 20: covid_data.loc[covid_data.index == i,'age'] = '11-20'
    if 21 <= age <= 30: covid_data.loc[covid_data.index == i,'age'] = '21-30'
    if 31 <= age <= 40: covid_data.loc[covid_data.index == i,'age'] = '31-40'
    if 41 <= age <= 50: covid_data.loc[covid_data.index == i,'age'] = '41-50'
    if 51 <= age <= 60: covid_data.loc[covid_data.index == i,'age'] = '51-60'
    if 61 <= age <= 70: covid_data.loc[covid_data.index == i,'age'] = '61-70'
    if age > 70: covid_data.loc[covid_data.index == i,'age'] = '70+'

In [0]:
a = pd.DataFrame(covid_data[covid_data['death'] == 1]).groupby('age').size().reset_index(name = 'a')
# sns.countplot(a)
a


In [0]:
fig = go.Figure().add_trace(go.Pie(values=a.a,labels= a.age,hole = .5))
fig.update_layout(margin=dict(l=0,r=20,b=0,t=50,pad=0),paper_bgcolor="white",title_text = 'Number of CoronaVirus Cases worldwide',)
fig.layout.hovermode = 'x'
fig.show()

###Weather Analysis

In [0]:
d = pd.read_csv('https://download.data.world/file_download/data-society/global-climate-change-data/GlobalLandTemperatures%2FGlobalLandTemperaturesByCountry.csv')

In [0]:
d

###Extra

In [0]:
import plotly.io as pio
pio.show(fig)#, file='index.html', auto_open=True)

In [11]:
!jupyter nbconvert --to html COVID_19_Analysis.ipynb

[NbConvertApp] Converting notebook COVID_19_Analysis.ipynb to html
[NbConvertApp] Writing 716711 bytes to COVID_19_Analysis.html


In [0]:
HTML("https://raw.githubusercontent.com/AbshkPskr/Portfolio/master/index.html") 